## **Configuring the account**

In [ ]:
import subprocess
from dotenv import load_dotenv
# identifying into github
!git config --global user.name "Nuzz23"
!git config --global user.email "nunzio.licalzi9@gmail.com"
output = subprocess.check_output("git config --global --list", shell=True).decode('utf-8').split()

# check if correctly identified
assert len(output) >= 2, "Wrong lenght"
assert output[0].split('=')[-1] == 'Nuzz23', 'wrong user name'
assert output[1].split('=')[-1] == 'nunzio.licalzi9@gmail.com', 'wrong email'
del output

load_dotenv()

## **Cloning the repository**

In [ ]:
# if the cloned repository already exists delete it
%cd /content
%rm -rf sample_data
if "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip():
  !rm -rf MLDL_SemanticSegmentation

/content


In [ ]:
# clone the repository via token
!git clone --quiet "{os.getenv('GITHUB_TOKEN')}"

# check if cloned correctly
assert "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip(), "Not cloned correctly"

/content/MLDL_SemanticSegmentation


# **LIBRARIES**

In [ ]:
# installing the required libraries
%cd /content/MLDL_SemanticSegmentation
%pip install -q -r requirements.txt

## **MAIN LIBRARIES**

In [ ]:
import torch, wandb

## **CUSTOM IMPORTS**

In [ ]:
from stats import countFLOPS, latency
from models.bisenet.build_bisenet import BiSeNet
from Extension.BiSeNetV2.model import BiSeNetV2
from train import mainBiSeNetBase
from datasets.downloader import Downloader

# **DATASET**

## **Downloading the dataset**

In [ ]:
if not Downloader().downloadCityScapes():
  raise FileNotFoundError("CityScapes dataset not found")

# **Model**

In [ ]:
def initBiSeNetOrV2Base(model_str:str='bisenet', totEpoches:int=50, width:int=1024, height:int=512, batchSize:int=4, momentum:float=0.9, learning_rate:float=0.0005,
                restartTraining:bool=False, pushWeights:bool=False, enablePrint:bool=False, enablePrintVal:bool=False):
    """
    Initializes the model and starts the training process.

    Args:
        totEpoches (int, optional): The total number of epochs to train the model. Defaults to 50.
        width (int, optional): The width of the input image. Defaults to 1024.
        height (int, optional): The height of the input image. Defaults to 512.
        batchSize (int, optional): The batch size to use for training. Defaults to 4.
        momentum (float, optional): The momentum to use for the optimizer. Defaults to 0.9.
        learning_rate (float, optional): The learning rate to use for the optimizer. Defaults to 0.005.
        restartTraining (bool, optional): Whether to restart the training process from scratch or use the weigths
            of the previous training epoch. Defaults to False, ie start from scratch the training.
        pushWeights (bool, optional): Whether to push the weights of the training to git. Defaults to False.
        enablePrint (bool, optional): Whether to enable print of the images during training. Defaults to False.
        enablePrintVal (bool, optional): Whether to enable print of the images during validation. Defaults to False.

    Returns:
        model (torch.nn.Module): The fully trained PyTorch model.
    """
    assert torch.cuda.is_available(), "Use cuda (T4 gpu not enabled)"
    wandb.login(key='53d3ed3ff56973b6eb203376a439b403344ea7d5')
    wandb.init()

    match model_str.lower() if isinstance(model_str, str) else model_str:
        case 'bisenetv2': model = BiSeNetV2(n_classes=19).cuda()
        case _: model, model_str = BiSeNet(num_classes=19, context_path='resnet18').cuda(), 'bisenet'

    if restartTraining:
        artifact = wandb.use_artifact(f'tempmailforme212-politecnico-di-torino/BiSeNetV2/model-weights:latest', type='model')
        artifact_dir = artifact.download()

        # Carica i pesi nel modello
        weights_path = f"{artifact_dir}/model_weights.pth"
        model.load_state_dict(torch.load(weights_path))

        print("Correctly loaded weights from the cloud of WandB!")

        starting_epoch = artifact.metadata['epoch']
    else:
        starting_epoch = 0

    wandb.init(project=f'BiSeNetV2',
                config={"starting_epoch": starting_epoch, "epoches":totEpoches, 'weight_decay':1e-4,
                        "learning_rate":learning_rate, "momentum":momentum,'batch_size':batchSize})

    return mainBiSeNetBase(wandb, model, model_str, width, height, pushWeights, enablePrint, enablePrintVal)


In [ ]:
WIDTH, HEIGHT = 1024, 512

model = initBiSeNetOrV2Base(model_str='bisenet', batchSize=4, learning_rate=0.0005, width=WIDTH, height=HEIGHT,
                   pushWeights=True, enablePrintVal=True, enablePrint=True, restartTraining=True)

In [ ]:
print('flops', countFLOPS(model, width=WIDTH, height=HEIGHT))

print('latencyMean, latencyStd, fpMean, fpsStd', *latency(model, width=WIDTH, height=HEIGHT))

print(f"Numero totale di parametri: {sum(p.numel() for p in model.parameters())}")

context_path.features.avgpool, context_path.features.fc, supervision1, supervision2


flops 25779775698
latencyMean, latencyStd, fpMean, fpsStd 15.839773893356325 2.301098004268971 64.58737348178171 13.600086919463912
Numero totale di parametri: 12581672


In [ ]:
from train import evaluateLastEpoch
evaluateLastEpoch(model)